# Import needed libraries

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.callbacks import TensorBoard
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import regularizers
from keras.layers import Dense, Embedding, LSTM, Conv1D, Conv2D, MaxPooling1D, Reshape, Flatten, Dropout, CuDNNLSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

import operator


# Import our dependencies
import tensorflow as tf

import tensorflow_hub as hub
import os
import re
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer


# Initialize session
sess = tf.Session()
K.set_session(sess)
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

#from appos import appos

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Start with loading all necessary libraries
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
% matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Loading, extracting and pre-processing the data

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth) 


link = 'https://drive.google.com/open?id=1B1BHZvMZ8BgfpLkPzsPz7sLGn8lubN7V'

fluff,id = link.split('=')
print (id)

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Reviews.csv')  
data = pd.read_csv('Reviews.csv')
data = data[['Text', 'Score']]



W0517 11:20:18.889568 140253575722880 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNo

1B1BHZvMZ8BgfpLkPzsPz7sLGn8lubN7V


# Convert the ratings to the sentiments negative and positive.
Each sentiment will be represented by integers 0 or 1.

0: negative

1: positive

The ratings and sentiments will be mapped as follows.

Rating 1-3 -> negative (0)

Rating 4-5 -> positive (1)

In [0]:


# Change from 1-5 ratings to negative or positive sentiment
rating_to_sentiment = { 1: 0, 2: 0, 3: 0, 4: 1, 5: 1 }
data['Sentiment'] = data['Score'].apply(lambda x: rating_to_sentiment[x])
data=data.sample(200000)

# Count number of negative and positive reviews
neg_num = pd.value_counts(data['Sentiment'])[0]
pos_num = pd.value_counts(data['Sentiment'])[1]

print('# negative reviews before: {}'.format(neg_num))
print('# positive reviews before: {}'.format(pos_num))

# Make the data set balanced
balanced_sample_num = np.min([neg_num, pos_num])

# Picks <'balanced_sample_num'> numbers of negative and positive reviews at random
data = (data.groupby('Sentiment', as_index = False)
        .apply(lambda x: x.sample(n = balanced_sample_num))
        .reset_index(drop = True))

# Shuffle the rows so that 0's and 1's are mixed
data = data.sample(frac = 1).reset_index(drop = True)


print('\n# negative reviews after: {}'.format(pd.value_counts(data['Sentiment'])[0]))
print('# positive reviews after: {}'.format(pd.value_counts(data['Sentiment'])[1]))
print(data['Text'])
# Get one-hot encoding for the labels
Y = pd.get_dummies(data['Sentiment']).values


# negative reviews before: 43800
# positive reviews before: 156200

# negative reviews after: 43800
# positive reviews after: 43800
0        My baby loves this formula. It does suck that ...
1        I love this cereal, and eat it daily.  I used ...
2        I am trying to find a little candy that is not...
3        I purchased this oil as I was thrilled that a ...
4        This kit arrived in a timely manner and has al...
5        The coins are hard, they take a day and a half...
6        Amazon needs to check their stock, I ordered t...
7        i was very disappointed in this soup.  it was ...
8        Because I like sweet potatoe fries, I thought ...
9        Buyer Beware Please! This sweetener is not for...
10       There is another brand of organic rooibos tea ...
11       This Product came with outter plastic cover re...
12       I started using Crisco on my skin about six mo...
13       If you want pumpking flavored coffee buy the T...
14       The people who sent the hot sauce

# Perform pre-processing on the text

In [0]:

# All characters to lower case
data['Text'] = data['Text'].apply(lambda x: x.lower())

# # Convert words with apostrophes to its corresponding words, e.g. "it's" -> "it is"
# data['Text'] = data['Text'].apply(lambda x: x.split())
# data['Text'] = data['Text'].apply(lambda x: " ".join([appos[word] if word in appos else word for word in x]))

# Remove html-tags, punctuation, commas, numbers etc
data['Text'] = data['Text'].apply((lambda x: re.sub('<[^<]+?>', ' ', x)))
data['Text'] = data['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', ' ', x)))
data['Text'] = data['Text'].apply((lambda x: re.sub('^\d+\s|\s\d+\s|\s\d+$', ' ', x)))

# Convert text into tokens, in this case sentences into words
data['Text'] = data.apply(lambda x: word_tokenize(x['Text']), axis = 1)

# Remove most commonly occuring words which are not relevant in the context of the data
irrelevant_words = stopwords.words('english')
data['Text'] = data['Text'].apply(lambda x: [word for word in x if word not in irrelevant_words])

# Find the base form of the word (lemmatization)
lemma = WordNetLemmatizer()
data['Text'] = data['Text'].apply(lambda x: " ".join([lemma.lemmatize(word) for word in x]))
print(data['Text'])


0        baby love formula suck store around sell order...
1        love cereal eat daily used buy vanilla creme v...
2        trying find little candy bad one aftertaste un...
3        purchased oil thrilled local grocer actually c...
4        kit arrived timely manner piece necessary buil...
5        coin hard take day half chew anise flavor lico...
6        amazon need check stock ordered arrived stale ...
7        disappointed soup thin oily vegetable little m...
8        like sweet potatoe fry thought would like foun...
9        buyer beware please sweetener everybody maltit...
10       another brand organic rooibos tea offered amaz...
11       product came outter plastic cover removed bar ...
12       started using crisco skin six month ago produc...
13       want pumpking flavored coffee buy timothy perf...
14       people sent hot sauce bbq sauce sent glass bot...
15       bought coffee work love unusual u like thing g...
16       treat made high quality ingredient made u two .

Split data-set into 2

In [0]:
"""smaller_set_percentage = 0.5
split = int(round(smaller_set_percentage * data.shape[0]))

X_small = data['Text'][0:split]
Y_small = Y[0:split]

X_large = data['Text'][split:data.shape[0]]
Y_large = Y[split:data.shape[0]]
print()
print('Small data set: X: {} Y: {}'.format(X_small.shape, Y_small.shape))
print('Large data set: X: {} Y: {}'.format(X_large.shape, Y_large.shape)) """

X=data
test_size, val_size = 0.2, 0.05

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = test_size)

print('X train: {}, Y train: {}'.format(X_train.shape, Y_train.shape))
print('X test: {}, Y test: {}'.format(X_test.shape, Y_test.shape))

train_x=X_train
test_x=X_test
train_y=Y_train
test_y=Y_test


"""smaller_set_percentage = 0.5
split = int(round(smaller_set_percentage * data.shape[0]))

train_x=data[split:]
test_x=data[:split]
train_y=Y[split:]
test_y=Y[:split]

print(train_x)
print(test_y)
print('training data set: X: {} Y: {}'.format(train_x.shape, train_y.shape))
print('testing data set: X: {} Y: {}'.format(test_x.shape, test_y.shape))"""

X train: (70080, 3), Y train: (70080, 2)
X test: (17520, 3), Y test: (17520, 2)


"smaller_set_percentage = 0.5\nsplit = int(round(smaller_set_percentage * data.shape[0]))\n\ntrain_x=data[split:]\ntest_x=data[:split]\ntrain_y=Y[split:]\ntest_y=Y[:split]\n\nprint(train_x)\nprint(test_y)\nprint('training data set: X: {} Y: {}'.format(train_x.shape, train_y.shape))\nprint('testing data set: X: {} Y: {}'.format(test_x.shape, test_y.shape))"

In [0]:
# Create a custom layer that allows us to update weights (lambda layers do not have trainable parameters!)

class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [0]:
!pip install allennlp

from allennlp.training.metrics.metric import Metric
from allennlp.training.metrics import f1_measure



     |████████████████████████████████| 5.4MB 32kB/s 
  Found existing installation: botocore 1.12.147
    Uninstalling botocore-1.12.147:
      Successfully uninstalled botocore-1.12.147


In [0]:
# Function to build model
!pip install keras_metrics
import keras
import keras_metrics
def build_model(): 
  input_text = layers.Input(shape=(1,), dtype="string")
  embedding = ElmoEmbeddingLayer()(input_text)
  dense = layers.Dense(256, activation='relu')(embedding)
  pred = layers.Dense(1, activation='sigmoid')(dense)

  model = Model(inputs=[input_text], outputs=pred)

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
 
  model.summary()
  #,keras_metrics.precision(), keras_metrics.recall()
  return model

In [0]:
# Create datasets (Only take up to 150 words for memory)

train_text = train_x['Text'].tolist()
train_text = [' '.join(t.split()[0:150]) for t in train_text]
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = train_x['Sentiment'].tolist()
print(train_text)

test_text = test_x['Text'].tolist()
test_text = [' '.join(t.split()[0:150]) for t in test_text]
test_text = np.array(test_text, dtype=object)[:, np.newaxis]
test_label = test_x['Sentiment'].tolist()

[['thick strong likely adding soup rice']
 ['wonderful product dog really love one thing make product better chew box would one every day month']
 ['movie always popular different entertaining fast moving hour half film available bllu ray look better nothing outstanding definite improvement shapnress several character mainly michael keaton unique sometimes revolting title character beetlejuice always fascinating watch whole movie also get lot humor scary special effect odd see alec baldwin low key role 90 played type guy davis look act like well davis many time played nice people viewer like took four viewing finally appreciated catharine hara comedic talent movie favorite someone find absolutely hilarious messed wife mother family move haunted house inhabited baldwin davis keaton made name actor whacked robin williams type role although never really followed anything popular film winona rider cute teenage daughter get fun supporting role diverse people talk show host dick cavett singe

In [0]:
# Build and fit
val_size=0.2
model = build_model()
model.fit(train_text, 
          train_label,
          validation_split=val_size,
          epochs=4,
          batch_size=32)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0517 14:44:49.580165 140253575722880 saver.py:1483] Saver not created because there are no variables in the graph to restore


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
elmo_embedding_layer_4 (Elmo (None, 1024)              4         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 257       
Total params: 262,661
Trainable params: 262,661
Non-trainable params: 0
_________________________________________________________________
Train on 56064 samples, validate on 14016 samples
Epoch 1/4
56064/56064 [==============================] - 1198s 21ms/step - loss: 0.4884 - acc: 0.7647 - val_loss: 0.4738 - val_acc: 0.7722
Epoch 2/4
56064/56064 [==============================] - 1191s 21ms/step - loss

In [0]:
""""model.save('ElmoModel.h5')
pre_save_preds = model.predict(test_text[0:100]) # predictions before we clear and reload model

# Clear and load model
model = None
model = build_model()
model.load_weights('ElmoModel.h5')

post_save_preds = model.predict(test_text[0:100]) # predictions after we clear and reload model
all(pre_save_preds == post_save_preds) # Are they the same?"""

'"model.save(\'ElmoModel.h5\')\npre_save_preds = model.predict(test_text[0:100]) # predictions before we clear and reload model\n\n# Clear and load model\nmodel = None\nmodel = build_model()\nmodel.load_weights(\'ElmoModel.h5\')\n\npost_save_preds = model.predict(test_text[0:100]) # predictions after we clear and reload model\nall(pre_save_preds == post_save_preds) # Are they the same?'

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [0]:
print(X_test)
Y_test_new=Y_test[:,1]
print(Y_test_new.shape)

                                                    Text  Score  Sentiment
45454  expecting coffee actually shipped jamaica plea...      5          1
25109  returning item came white baggie box packaging...      2          0
9629   like tea strong tea delivers let brew longer 7...      5          1
9799   think sitting around warehouse long time took ...      1          0
32012  friend bought tea tjmaxx box contained bag fin...      5          1
8932   beagle lab mix fairly picky treat go nut tryin...      5          1
50203  ordered product expecting receive whole seed p...      1          0
62542  nice able buy quality organic sugar good price...      5          1
77138  used drink time great coffee recently noticed ...      2          0
35540  title review suggests thought jerky little rub...      3          0
63826  big fan emerald mixed nut plastic lately offer...      1          0
64228  coffee tasted bitter like burnt cleaned machin...      1          0
39074  tea good smell yum

In [0]:

# predict probabilities for test set
yhat_probs = model.predict(X_test['Text'], verbose=1)
# predict crisp classes for test set
labels = (yhat_probs > 0.5).astype(np.int)
print(labels)

#yhat_classes = np.argmax(yhat_probs,axis=1)
# accuracy: (tp + tn) / (p + n)



 1696/17520 [=>............................] - ETA: 6:24

ResourceExhaustedError: ignored

In [0]:
print(yhat_probs)
print(Y_test_new)

In [0]:

#test_label1= np.array(test_label)
#print(test_label1)
#print(test_label1.shape)
#Y_test_new=test_label1[:,1]

print("elmo_model")
accuracy = accuracy_score(Y_test_new,labels)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(Y_test_new, labels)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(Y_test_new,labels)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Y_test_new,labels)
print('F1 score: %f \n\n' % f1)